# Lab Wednesday November 30

Please put the name and student ID of each group member here:


# Review

## Linear Programming
**Linear programming**
* the objective function is linear
* the feasible set is defined by linear equations or inequalities, 

The general form:

\begin{align}
  \min_x & &  c^Tx \\	
\text{subject to} & & A_{ub}x\leq b_{ub}\\
  & & A_{eq}x = b_{eq}\\
  & & l\leq x\leq u
\end{align}



## The linprog function in scipy.optimize

* Usage: `linprog(c, A_ub, b_ub, A_eq, b_eq, bounds)`:
  * If a corresponding constraint does not exist, pass `None` to the corresponding argument
  * bounds: A sequence of (min, max) pairs for each element in x, defining the minimum and maximum values of that decision variable. Use None to indicate that there is no bound. By default, bounds are (0, None) (all decision variables are non-negative). If a single tuple (min, max) is provided, then min and max will serve as bounds for all decision variables. 
* Return value: an object containing at least the following instance varibles
* `x`: The values of the decision variables that minimizes the objective function while satisfying the constraints.
* `fun`: The optimal value of the objective function $c\dot x$.
* `success`: True when the algorithm succeeds in finding an optimal solution.
* `status`: An integer representing the exit status of the algorithm.
  * 0 : Optimization terminated successfully.
  * 1 : Iteration limit reached.
  * 2 : Problem appears to be infeasible.
  * 3 : Problem appears to be unbounded.
  * 4 : Numerical difficulties encountered.
* `message`: A string descriptor of the exit status of the algorithm.


# Example

Suppose that a machine can produce 100 cases of six-ounce glasses in 6 hours, or 100 cases of ten-ounce cocktail glasses in 5 hours. The scheduled production time is 60 hours per week. The available storage is 15,000 cubic feet. A case of six-ounce glasses requires 10 cubic feet of storage space, while a case of ten-ounce glasses requires 20 cubic feet. The profit of the six-ounce glasses is CAD5.00 per case; however, there is a market capacity of 800 cases per week. The profit of the ten-ounce glasses is CAD4.50 per case and there is no market capacity. How many cases of each type of glass should be produced each week in order to maximize the total prodit?

**Soluation** Let $x$ and $y$ be the number of six- and ten-ounce glasses to be produced per week.

* **Maximizing the profit**
$$ \max_{x,y} 5x+4.5y $$

* **Constrains**
  * Total production time:
  $$ 6x/100 + 5y/100 \leq 60 $$
  * Total storage:
  $$ 10x+20y\leq 15000 $$
  * Bounds:
  $$ 0\leq x\leq 800,\;\text{ and }0\leq y $$

## The arguments of the linprog function
* the objective function's coefficients: This is a maximization problem, we need to convert it to a minimization problem
   * maximizing $f(x)$ is equivalent to minimizing $-f(x)$
   * $c=[-5, -4.5]$
* the constrains:
  * The upper bounds:
    * Note that all inequalities must be written as $A_u x\leq b_u$
    * Here
    $$ A_{ub} = \left[\begin{array}{cc} 0.06 & 0.05 \\ 10 & 20\end{array}\right], \;
       b_{ub} = \left[\begin{array}{cc} 60\\ 15000\end{array}\right]$$
       
  * The equation constraints: None
    * `A_eq=None`
    * `b_eq=None`
  * The bounds:
    * For $x$: `[0, 800]`
    * For $y$: `[0, None]`


In [20]:
from scipy.optimize import linprog
from numpy import array
c = array([-5, -4.5])
A_ub = array([[0.06, 0.05], [10, 20]])
b_ub = array([60, 15000])

linprog(c, A_ub = A_ub, b_ub = b_ub, A_eq = None, b_eq = None, bounds = [[0, 800], [0, None]])

     con: array([], dtype=float64)
     fun: -5142.857141392633
 message: 'Optimization terminated successfully.'
     nit: 6
   slack: array([1.70755143e-08, 4.30018554e-06])
  status: 0
 success: True
       x: array([642.85714268, 428.57142845])

The maximum profit is CAD -(-5142.86)=5142.86, by producting 642.86 6-ounce boxed and 428.57 10-ounce boxes.

## Integer solutions
However, we cannot produce a fraction of a case. So we need to choose from 3 possible answers:
* $x=642$, $y=428$
  * This solution must be in the feasible region
* $x=643$, $y=428$
  * We need to check if this is in the feasible region
* $x=642$, $y=429$
  * We need to check if this is in the feasible region
Note that $x=643$ and $y=429$ must be out of the feasible region (why?)

In [10]:
from numpy import matmul
sol1 = array([642, 428])
-sum(c*sol1)


5136.0

In [12]:
sol2 = array([643, 428])
print("profit=", -sum(c*sol2))
matmul(A_ub, sol2) < b_ub

profit= 5141.0


array([ True,  True])

In [13]:
sol2 = array([642, 429])
print("profit=", -sum(c*sol2))
matmul(A_ub, sol2) < b_ub

profit= 5140.5


array([ True, False])

Thus, $x=643$, $y=428$ is still in the feasible region, the maximum profit is CAD 5141.
 

# Group assignment

A company manufactures three lines of products: the basic, the pro and the premium, with respective profits $3, $9, and $25. respectively. The distribution center requires that at least 250 basic, 375 pro, and 150 premium products be produced each week. Each product requires a certain amount of time in order to: (1) manufacture the body; (2) assemble the parts; and (3) inspect, test, and package. The basic takes 0.1 hours to manufacture, 0.2 hours to assemble, and 0.1 hours to inspect, test, and package. The pro needs 0.2, 0.35, and 0.2 hours respectively, and the premium requires 0.7, 0.1, and 0.3 hours, respectively. In addition, there are 250 hours per week of manufacturing time available, 350 hours of assembly, and 150 hours total to inspect, test, and package.

How many products of each type should be produced to maximize the profit? Your answer may contain fractions, there is no need to find an integer answer.

## Solution

Let $x$, $y$ and $z$ be the number of products in each line be produced.

We want maximize the profit: 
$$\max_{x,y,z} 3x+9y+25z $$
This is equivalent to
$$\min_{x,y,z} -3x-9y-25z $$

Constraints:
* Distribution center (these are bounds)
  * Basic: $x\geq 250$
  * Pro: $y\geq375$
  * Premium: $z\geq 150 $
* Manufacture time (these are upper bounds)
  * Basic: $0.1x+0.2y+0.7z\leq 250$
  * Pro: $0.2x + 0.35 y + 0.1z\leq 350$
  * Premium: $0.1x + 0.2y+0.3z\leq 150$


In [22]:
from scipy.optimize import linprog
from numpy import array
c = array([-3, -9, -25])
A_ub = array([[0.1, 0.2, 0.7], [0.2, 0.35, 0.1], [0.1, 0.2, 0.3]])
print(A_ub)
b_ub = array([250, 350, 150])
bounds = [[250, None], [375, None], [150, None]]
linprog(c, A_ub = A_ub, b_ub = b_ub, A_eq = None, b_eq = None, bounds = bounds)

[[0.1  0.2  0.7 ]
 [0.2  0.35 0.1 ]
 [0.1  0.2  0.3 ]]


     con: array([], dtype=float64)
     fun: -8291.666666663641
 message: 'Optimization terminated successfully.'
     nit: 6
   slack: array([3.33333333e+01, 1.52083333e+02, 3.58397756e-11])
  status: 0
 success: True
       x: array([250.        , 375.        , 166.66666667])

Thus, the optimal answer is to produce 250 basics, 375 pros and 166 premiums. 